# 1. Test Random Environment with OpenAI Gym

In [1]:
import random
import gym

In [2]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [3]:
states

4

In [4]:
actions

2

In [5]:
episodes = 10
for episode in range(1, episodes + 1): 
    state = env.reset()
    done = False
    score = 0
    
    while not done:
#         env.render()
        action = random.choice([0, 1]) # random step, 0 and 1 correspond to left and right
        n_state, reward, done, info = env.step(action)
        score += reward
        
    print(f"Episode: {episode}, Score: {score}")
        

Episode: 1, Score: 33.0
Episode: 2, Score: 21.0
Episode: 3, Score: 17.0
Episode: 4, Score: 11.0
Episode: 5, Score: 29.0
Episode: 6, Score: 30.0
Episode: 7, Score: 13.0
Episode: 8, Score: 27.0
Episode: 9, Score: 37.0
Episode: 10, Score: 11.0


# 2. Create DL Model with Keras

In [17]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [18]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [19]:
model = build_model(states, actions)

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [10]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [11]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(
        model=model, memory=memory, policy=policy, nb_actions=actions,
        nb_steps_warmup=10, target_model_update=1e-2
    )
    return dqn

In [21]:
dqn = build_agent(model, actions)

In [22]:
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 8:28 - reward: 1.0000

/Users/nathanieldake/development/data-science/intuitiveml/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/Users/nathanieldake/development/data-science/intuitiveml/venv/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000
103 episodes - episode_reward: 96.252 [9.000, 200.000] - loss: 3.503 - mae: 19.709 - mean_q: 39.870

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 42s 4ms/step - reward: 1.0000
50 episodes - episode_reward: 197.840 [183.000, 200.000] - loss: 8.866 - mae: 43.095 - mean_q: 86.948

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 43s 4ms/step - reward: 1.0000
51 episodes - episode_reward: 198.843 [183.000, 200.000] - loss: 7.530 - mae: 46.832 - mean_q: 94.258

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: 1.0000
50 episodes - episode_reward: 199.080 [185.000, 200.000] - loss: 4.939 - mae: 43.874 - mean_q: 88.307

Interval 5 (40000 steps performed)
 1702/10000 [====>.........................] - ETA: 36s - reward: 1.0000done, took 182.117 seconds
